In [1]:
import numpy as np
import keras
from keras.models import  load_model
import argparse, sys
import matplotlib.pyplot as plt

import os
import csv
#import torch
import pandas as pd
import numpy as np
import re
import itertools

from itertools import groupby
import scipy.sparse.linalg as spl
from scipy.cluster.hierarchy import *
%matplotlib inline

Using TensorFlow backend.


In [23]:
"""
Get the unsupervised representations
"""

num_components = 5 # Number of components to use after PCA

def readComponent(comps):
    namelist = []
    numlist = []
    ccomps = comps
    while(len(ccomps) != 0):
        stemp = ccomps[1:]
        if(len(stemp) == 0):
            namelist.append(ccomps)
            numlist.append(1.0)
            break
        it = 0
        for st in stemp:
            it = it + 1
            if(st.isupper()):
                im = 0
                for mt in stemp[:it]:
                    im = im + 1
                    if(mt.isdigit()):
                        namelist.append(ccomps[0:im])
                        numlist.append(float(ccomps[im:it]))
                        ccomps = ccomps[it:]
                        break
                    elif(im == len(stemp[:it])):
                        namelist.append(ccomps[0:im])
                        numlist.append(1.0)
                        ccomps = ccomps[it:]
                        break
                break
            elif(it == len(stemp)):
                im = 0
                for mt in stemp:
                    im = im + 1
                    if(mt.isdigit()):
                        namelist.append(ccomps[0:im])
                        numlist.append(float(ccomps[im:]))
                        ccomps = ccomps[it+1:]
                        break
                    elif(im == len(stemp)):
                        namelist.append(ccomps)
                        numlist.append(1.0)
                        ccomps = ccomps[it+1:]
                        break
                break
    return namelist, numlist

def get_atom_environment_pair(compound):
    namelist, numlist = readComponent(compound)
    atom_environment_pair = []
    for i, element in enumerate(namelist):
        environment = str(numlist[i])
        for j, other_elem in enumerate(namelist):
            if other_elem != element:
                environment += other_elem + str(numlist[j])

        atom_environment_pair.append((element, environment))    
    return atom_environment_pair

atom_environment_pairs = []
num_compounds = 0

#f = np.load('data/tc_mat_clean2.npy')
with open('data/feature_learn.csv', 'r') as f:
    reader = csv.reader(f)
    for line in reader:
        compound = line[0]
        #print(compound)
        atom_environment_pair = get_atom_environment_pair(compound)
        atom_environment_pairs.extend(atom_environment_pair)
        num_compounds += 1
#for line in f:
#    compound, target = line#.split()
#    atom_environment_pair = get_atom_environment_pair(compound)
#    atom_environment_pairs.extend(atom_environment_pair)
#    num_compounds += 1

environment_set = set()
element_set = set()

for atom, environment in atom_environment_pairs:
    element_set.add(atom)
    environment_set.add(environment)

environment_set = list(environment_set)
element_set = list(element_set)
print("Num Elements: {}, Num Environments: {}".format(len(element_set), len(environment_set)))

atom_env = np.zeros((len(element_set), len(environment_set)))

p = 2.0
for idx1, ele in enumerate(element_set):
    for idx2, env in enumerate(environment_set):
        env_split = [''.join(x) for _, x in groupby(env, str.isalpha)]
        if ele in env_split:
            atom_env[idx1][idx2] = 1.
    norm = np.sum(atom_env[idx1]**p)**(1./p)
    atom_env[idx1]=atom_env[idx1]/norm

def SVD(m):
    u, s, vt = spl.svds(m, k=min(m.shape)-1)
    return u, s, vt

def PCA(m, k=10):
    u, s, vt = SVD(m)
    pc = np.matmul(u[:, :k], np.diag(s)[:k, :k])
    pa = vt[:k, :]
    rx = np.matmul(pc, pa)
    return pc, pa, rx

f_matrix, v_matrix, red_atom_env = PCA(atom_env, num_components)

elem2idx = {}
for i, elem in enumerate(element_set):
    elem2idx[elem] = i

Num Elements: 86, Num Environments: 83034


In [24]:
"""
Creating the representation
"""

ChemicalSymbols = [ 'X',  'H',  'He', 'Li', 'Be','B',  'C',  'N',  'O',  'F',
                    'Ne', 'Na', 'Mg', 'Al', 'Si','P',  'S',  'Cl', 'Ar', 'K',
                    'Ca', 'Sc', 'Ti', 'V',  'Cr','Mn', 'Fe', 'Co', 'Ni', 'Cu',
                    'Zn', 'Ga', 'Ge', 'As', 'Se','Br', 'Kr', 'Rb', 'Sr', 'Y',
                    'Zr', 'Nb', 'Mo', 'Tc', 'Ru','Rh', 'Pd', 'Ag', 'Cd', 'In',
                    'Sn', 'Sb', 'Te', 'I',  'Xe','Cs', 'Ba', 'La', 'Ce', 'Pr',
                    'Nd', 'Pm', 'Sm', 'Eu', 'Gd','Tb', 'Dy', 'Ho', 'Er', 'Tm',
                    'Yb', 'Lu', 'Hf', 'Ta', 'W','Re', 'Os', 'Ir', 'Pt', 'Au',
                    'Hg', 'Tl', 'Pb', 'Bi', 'Po','At', 'Rn', 'Fr', 'Ra', 'Ac',
                    'Th', 'Pa', 'U',  'Np', 'Pu','Am', 'Cm', 'Bk', 'Cf', 'Es',
                    'Fm', 'Md', 'No', 'Lr']

atomicPeriod = [0, 1, 1, 2, 2, 2, 2, 2, 2, 2,
                2, 3, 3, 3, 3, 3, 3, 3, 3, 4,
                4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
                4, 4, 4, 4, 4, 4, 4, 5, 5, 5,
                5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
                5, 5, 5, 5, 5, 6, 6, 6, 6, 6,
                6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
                6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
                6, 6, 6, 6, 6, 6, 6, 7, 7, 7,
                7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
                7, 7, 7, 7]

atomicGroup = [0,  1, 18,  1,  2, 13, 14, 15, 16, 17,
              18,  1,  2, 13, 14, 15, 16, 17, 18,  1,
               2,  3,  4,  5,  6,  7,  8,  9, 10, 11,
              12, 13, 14, 15, 16, 17, 18,  1,  2,  3,
               4,  5,  6,  7,  8,  9, 10, 11, 12, 13,
              14, 15, 16, 17, 18,  1,  2,  3,  3,  3,
               3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
               3,  3,  4,  5,  6,  7,  8,  9, 10, 11,
              12, 13, 14, 15, 16, 17, 18,  1,  2,  3,
               3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
               3,  3,  3,  3]

atomicNum = {}
for anum, symbol in enumerate(ChemicalSymbols):
    atomicNum[symbol] = anum

class atom:
    def __init__(self, symbol, num=1):
        self.name = symbol
        self.atomicNum = atomicNum[symbol]
        self.atomicPeriod = atomicPeriod[atomicNum[symbol]]
        self.atomicGroup = atomicGroup[atomicNum[symbol]]
        self.num = num
        elem_idx = elem2idx[symbol]
        self.element_vec = f_matrix[elem_idx, :]

def get_component_vector(comps):
    namelist, numlist = readComponent(comps)
    avector = np.zeros((10, 10, num_components + 1))
    asum = sum(numlist)

    for i in range(len(namelist)):
        at = atom(namelist[i])
        idx = at.atomicNum - 1
        r_idx = idx // 10
        c_idx = idx % 10
        avector[r_idx, c_idx, 0] = numlist[i] / asum
        avector[r_idx, c_idx, 1:] = at.element_vec

    return avector

In [25]:
"""
Training
"""

X, y = [], []

#f = np.load('data/tc_mat_clean2.npy')
#for line in f:
with open('data/feature_learn.csv', 'r') as f:
    reader = csv.reader(f)
    for line in reader:
        compound, target = line
        namelist, numlist = readComponent(compound)
        X.append(get_component_vector(compound))
        y.append(target)

X = np.stack(X)
y = np.stack(y)

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
import keras
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization 
from keras.layers.core import Dropout, Flatten, Dense, Activation

def ATCNN_Tc_model():

    input_shape = (10,10,6)
    model = Sequential()

    # layer1 
    model.add(Conv2D(64, kernel_size=(5, 5), padding='same',input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    # layer2
    model.add(Conv2D(64, kernel_size=(3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    # layer3
    model.add(Conv2D(64, kernel_size=(3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    # layer4
    model.add(Conv2D(64, kernel_size=(3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    # layer5
    model.add(Conv2D(64, kernel_size=(2, 2), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    # layer6
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Flatten())
    model.add(Dense(200))

    # layer7
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    
    model.add(Dense(100))

    # layer8
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Activation('relu'))

    # layer9
    model.add(Dense(1))
    model.add(Activation('linear'))
    

    model.compile(loss=keras.losses.mean_absolute_error,
            optimizer= keras.optimizers.Adadelta())
    return model

In [28]:
model = ATCNN_Tc_model()
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 10, 10, 64)        9664      
_________________________________________________________________
batch_normalization_9 (Batch (None, 10, 10, 64)        256       
_________________________________________________________________
activation_10 (Activation)   (None, 10, 10, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 10, 10, 64)        36928     
_________________________________________________________________
batch_normalization_10 (Batc (None, 10, 10, 64)        256       
_________________________________________________________________
activation_11 (Activation)   (None, 10, 10, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 10, 10, 64)       

In [29]:
model.fit(X_train, y_train, validation_split=0.02, batch_size=128, epochs=100)
model.save('model1.h5')


Train on 19286 samples, validate on 394 samples
Epoch 1/100
19286/19286 [==============================] - 9s 483us/step - loss: 13.5379 - val_loss: 19.6030
Epoch 2/100
19286/19286 [==============================] - 8s 404us/step - loss: 7.0999 - val_loss: 19.2739
Epoch 3/100
19286/19286 [==============================] - 8s 405us/step - loss: 6.5658 - val_loss: 14.8570
Epoch 4/100
19286/19286 [==============================] - 8s 405us/step - loss: 6.2868 - val_loss: 6.0517
Epoch 5/100
19286/19286 [==============================] - 8s 407us/step - loss: 5.9699 - val_loss: 6.4558
Epoch 6/100
19286/19286 [==============================] - 8s 403us/step - loss: 5.9339 - val_loss: 5.2158
Epoch 7/100
19286/19286 [==============================] - 8s 410us/step - loss: 5.7398 - val_loss: 5.7915
Epoch 8/100
19286/19286 [==============================] - 8s 417us/step - loss: 5.5865 - val_loss: 5.1861
Epoch 9/100
19286/19286 [==============================] - 8s 423us/step - loss: 5.5578 - va

Epoch 77/100
19286/19286 [==============================] - 25s 1ms/step - loss: 3.7634 - val_loss: 4.6235
Epoch 78/100
19286/19286 [==============================] - 25s 1ms/step - loss: 3.7881 - val_loss: 4.4140
Epoch 79/100
19286/19286 [==============================] - 26s 1ms/step - loss: 3.7783 - val_loss: 4.6145
Epoch 80/100
19286/19286 [==============================] - 26s 1ms/step - loss: 3.8103 - val_loss: 4.7715
Epoch 81/100
19286/19286 [==============================] - 26s 1ms/step - loss: 3.7328 - val_loss: 4.4326
Epoch 82/100
19286/19286 [==============================] - 25s 1ms/step - loss: 3.7545 - val_loss: 4.2018
Epoch 83/100
19286/19286 [==============================] - 25s 1ms/step - loss: 3.7309 - val_loss: 4.2983
Epoch 84/100
19286/19286 [==============================] - 25s 1ms/step - loss: 3.6917 - val_loss: 4.8463
Epoch 85/100
19286/19286 [==============================] - 25s 1ms/step - loss: 3.7714 - val_loss: 4.5046
Epoch 86/100
19286/19286 [===========

In [30]:
loss = model.evaluate(X_test, y_test, batch_size=128)
y_calc = model.predict(X_test, batch_size=128)
print('test set loss:',loss)

4921/4921 [==============================] - 2s 394us/step
test set loss: 4.028872994140163


In [32]:
def atom_name(compound):
    elem_set = []
    dec = [''.join(x) for _, x in groupby(compound, str.isalpha)]
    for i in dec:
        try: 
            float(i)
        except:  
            for j in re.findall('[A-Z][^A-Z]*', i):
                elem_set.append(j)
    return elem_set

In [33]:
fe_based_sc_x = []
fe_based_sc_y = []
cu_based_sc_x = []
cu_based_sc_y = []
bcs_sc_x = []
bcs_sc_y = []
others_sc_x = []
others_sc_y = []

f = np.load('data/tc_mat_clean2.npy')
ax = f[:,0]
ay = f[:,1]

ax_train, ax_test, ay_train, ay_test = train_test_split(ax, ay, test_size=0.2, random_state=42)

for compound, target in zip(ax_test, ay_test):
    #print(compound,target)
    nm = atom_name(compound)
    
    if 'Fe' in nm and 'Cu' in nm:
        others_sc_x.append(get_component_vector(compound))
        others_sc_y.append(target)
    elif 'Fe' in nm:
        fe_based_sc_x.append(get_component_vector(compound))
        fe_based_sc_y.append(target)
    elif 'Cu' in nm:
        cu_based_sc_x.append(get_component_vector(compound))
        cu_based_sc_y.append(target)
    elif 'Fe' not in nm and 'Cu' not in nm:
        bcs_sc_x.append(get_component_vector(compound))
        bcs_sc_y.append(target)    

In [34]:
fe_based_sc_x = np.asarray(fe_based_sc_x)
fe_based_sc_y = np.asarray(fe_based_sc_y)
cu_based_sc_x = np.asarray(cu_based_sc_x)
cu_based_sc_y = np.asarray(cu_based_sc_y)
bcs_sc_x = np.asarray(bcs_sc_x)
bcs_sc_y = np.asarray(bcs_sc_y)
others_sc_x = np.asarray(others_sc_x)
others_sc_y = np.asarray(others_sc_y)

In [39]:
print(fe_based_sc_x.shape)
print(cu_based_sc_x.shape)
print(bcs_sc_x.shape)
print(others_sc_x.shape)

(282, 10, 10, 6)
(1156, 10, 10, 6)
(1215, 10, 10, 6)
(67, 10, 10, 6)


In [36]:
loss1 = model.evaluate(fe_based_sc_x, fe_based_sc_y, batch_size=128)
calc1 = model.predict(fe_based_sc_x, batch_size=128)
loss2 = model.evaluate(cu_based_sc_x, cu_based_sc_y, batch_size=128)
calc2 = model.predict(cu_based_sc_x, batch_size=128)
loss3 = model.evaluate(bcs_sc_x, bcs_sc_y, batch_size=128)
calc3 = model.predict(bcs_sc_x, batch_size=128)
loss4 = model.evaluate(others_sc_x, others_sc_y, batch_size=128)
calc4 = model.predict(others_sc_x, batch_size=128)

67/67 [==============================] - 0s 506us/step


In [37]:
print(loss2)
print(loss1)
print(loss4)
print(loss3)


8.873446259944084
3.7606631441319243
10.76606559753418
1.2371067647580747


In [38]:
prd = ['Hg1', 'Mg1B2', 'Fe1Se1', 'Y1Ba2Cu3O7']
px = []
for i in prd:
    px.append(get_component_vector(i))
model.predict(np.asarray(px), batch_size=128)

array([[ 3.2431262],
       [36.85376  ],
       [ 4.9418726],
       [85.01143  ]], dtype=float32)